### Project Description

The solar calculator is [Apogee Instrument's Clear Sky Calculator](http://clearskycalculator.com/quantumsensor.htm).

The day length is calculated from a Excel Workbook from NOAA. The original is located at [NOAA's Solar Calculations](https://www.esrl.noaa.gov/gmd/grad/solcalc/NOAA_Solar_Calculations_year.xls).

### Libraries
Import relevant libraries

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import math
import numpy as np
import pandas as pd

import datetime as dt
from datetime import datetime, timedelta, timezone
from dateutil import tz

import webbrowser

# import seaborn as sns
# import matplotlib.pyplot as plt
# %matplotlib inline

### Location and General Settings
Choose latitude and longitude location.

In [ ]:
Latitude = 40.7
Longitude = -111.9 # - to W for Sunrise/Sunset Calculator and + to W for Apogee Calulator (code adjusts for this)

If Latitude and Longitude are unknown can use the webpage below.

In [ ]:
url = 'https://itouchmap.com/?r=latlong'
webbrowser.open(url)

Local UTC offset (+ to East) and year for day durations.

In [ ]:
utc = -7
year = 2020
local_time = '12:00:00'

If UTC offset is unkown use the webpage below.

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_UTC_time_offsets'
webbrowser.open(url)

If you desire to see daylight savings time enter yes or no and the dates below.

In [ ]:
daylight_savings = 'No'
start_day = '2020-01-02' # add after
end_day = '2020-11-01'

Elevation (m) and longitude of time zone. For more info see [here](http://clearskycalculator.com/longitudeTZ.htm).

In [ ]:
Longitude_tz = 105
Elevation = 1291

If Longitude_tz is unknown use the webpage below.

In [ ]:
url = 'http://clearskycalculator.com/longitudeTZ.htm'
webbrowser.open(url)

Choose location name for filename.

In [ ]:
location = 'LoganUT'

Select if hourly average climate values are being used.

In [ ]:
hourly_averages = 'Yes'
hourly_averages_filename = 'Hourly_Averages_SaltLake' # Must be csv file
hourly_location_averages = 'SaltLakeCity'

Select if daily average climate values are being used (currently unsupported and being developed).

In [ ]:
# daily_averages = 'No'
# daily_averages_filename = 'Daily_Averages_Logan' # Must be csv file
# daily_location_averages = 'Logan'

If looking for averages to download (both hourly and daily). These will contain the proper formatting for the code below.

In [ ]:
url = 'https://www.ncdc.noaa.gov/cdo-web/search'
webbrowser.open(url)

### Functions to Run
Functions relevant to model

In [ ]:
def time_to_decimal(time):
    # Takes in a time format as a string such as 12:00:00
    # Returns a decimal time
    x = time.split(":")
    h = int(x[0])/24
    m = int(x[1])/(24*60)
    s = int(x[2])/(24*60*60)
    time = h + m + s
    return time

In [ ]:
def decimal_to_time(time):
    # Takes in a decimal time format
    # Returns a string time such as 12:00:00
    time = time * 24
    hours = int(time)
    minutes = (time*60) % 60
    seconds = (time*3600) % 60
      
    return "%d:%02d:%02d" % (hours, minutes, seconds)

In [ ]:
def tz_offset(time, offset):
    time = time.split(":")
    hour = int(time[0]) + offset
    return "%d:%02d:%02d" % (hour, int(time[1]), int(time[2]))

In [ ]:
def Hourly_Normals(date_time, hourly_averages, weather_model):
    # Must be called after creating webdriver object and import hourly normals
    
    #Get 30 Year Average Values
    Temp_90th = hourly_averages.loc[date_time]['HLY-TEMP-90PCTL']
    Temp_mean = hourly_averages.loc[date_time]['HLY-TEMP-NORMAL']
    Temp_10th = hourly_averages.loc[date_time]['HLY-TEMP-10PCTL']
    Dew_90th = hourly_averages.loc[date_time]['HLY-DEWP-90PCTL']
    Dew_Mean = hourly_averages.loc[date_time]['HLY-DEWP-NORMAL']
    Dew_10th = hourly_averages.loc[date_time]['HLY-DEWP-10PCTL']
    Clouds_Broken = hourly_averages.loc[date_time]['HLY-CLOD-PCTBKN']
    Clouds_Clear = hourly_averages.loc[date_time]['HLY-CLOD-PCTCLR']
    Clouds_Scattered = hourly_averages.loc[date_time]['HLY-CLOD-PCTSCT']
    Clouds_Overcast = hourly_averages.loc[date_time]['HLY-CLOD-PCTOVC']
    Clouds_Few = hourly_averages.loc[date_time]['HLY-CLOD-PCTFEW']

    #Calculate relative humidity
    Rel_Hum = 100*(math.exp((17.625*Dew_Mean)/(243.04+Dew_Mean))/math.exp((17.625*Temp_mean)/(243.04+Temp_mean)))

    #Calculate Estimated PPF
    Time_Input.send_keys(Keys.CONTROL,"a")
    Time_Input.send_keys(str(hour))
    Air_Temperature_Input.send_keys(Keys.CONTROL,"a")
    Air_Temperature_Input.send_keys(str(Temp_mean))
    Relative_Humidity.send_keys(Keys.CONTROL,"a")
    Relative_Humidity.send_keys(str(Rel_Hum))
    Recalculate.click()
    Est_PPF = Estimated_PPF.get_attribute('value')
    
    #Add row to weather model dataframe
    weather_model.loc[len(weather_model.index)] = [date_time,Light,Dark,Est_PPF,Temp_90th,Temp_mean,Temp_10th,Dew_90th,Dew_Mean,Dew_10th,Clouds_Broken,Clouds_Clear,Clouds_Scattered,Clouds_Overcast,Clouds_Few,Rel_Hum]

In [ ]:
def Hourly_Normals_Offset(date_time, hourly_averages, weather_model, offset):
    # Used to fill holes in data by creating daily of hourly offsets and assuming weather normals will be similar
    
    # Must be called after creating webdriver object and import hourly normals
    date_time_off = date_time + offset 
    
    #Get 30 Year Average Values
    Temp_90th = hourly_averages.loc[date_time_off]['HLY-TEMP-90PCTL']
    Temp_mean = hourly_averages.loc[date_time_off]['HLY-TEMP-NORMAL']
    Temp_10th = hourly_averages.loc[date_time_off]['HLY-TEMP-10PCTL']
    Dew_90th = hourly_averages.loc[date_time_off]['HLY-DEWP-90PCTL']
    Dew_Mean = hourly_averages.loc[date_time_off]['HLY-DEWP-NORMAL']
    Dew_10th = hourly_averages.loc[date_time_off]['HLY-DEWP-10PCTL']
    Clouds_Broken = hourly_averages.loc[date_time_off]['HLY-CLOD-PCTBKN']
    Clouds_Clear = hourly_averages.loc[date_time_off]['HLY-CLOD-PCTCLR']
    Clouds_Scattered = hourly_averages.loc[date_time_off]['HLY-CLOD-PCTSCT']
    Clouds_Overcast = hourly_averages.loc[date_time_off]['HLY-CLOD-PCTOVC']
    Clouds_Few = hourly_averages.loc[date_time_off]['HLY-CLOD-PCTFEW']

    #Calculate relative humidity
    Rel_Hum = 100*(math.exp((17.625*Dew_Mean)/(243.04+Dew_Mean))/math.exp((17.625*Temp_mean)/(243.04+Temp_mean)))

    #Calculate Estimated PPF
    Time_Input.send_keys(Keys.CONTROL,"a")
    Time_Input.send_keys(str(hour))
    Air_Temperature_Input.send_keys(Keys.CONTROL,"a")
    Air_Temperature_Input.send_keys(str(Temp_mean))
    Relative_Humidity.send_keys(Keys.CONTROL,"a")
    Relative_Humidity.send_keys(str(Rel_Hum))
    Recalculate.click()
    Est_PPF = Estimated_PPF.get_attribute('value')
    
    #Add row to weather model dataframe
    weather_model.loc[len(weather_model.index)] = [date_time,Light,Dark,Est_PPF,Temp_90th,Temp_mean,Temp_10th,Dew_90th,Dew_Mean,Dew_10th,Clouds_Broken,Clouds_Clear,Clouds_Scattered,Clouds_Overcast,Clouds_Few,Rel_Hum]

### Solar Position Calculator
Calculates sunrise/sunset and other solar position values based on Astromonomical Algortithms by Jean Meeus. Formulas were adapted from a National Oceanic and Atomospheric Adminstration [Excel sheet](https://www.esrl.noaa.gov/gmd/grad/solcalc/calcdetails.html). Further details can be found in the Github README or at the link.

In [ ]:
# Create dataframe
sol = pd.DataFrame(columns=['Date'])

# Create date range index
start = str(year)+'-01-01'
end = str(year)+'-12-31'
start = datetime.strptime(start, '%Y-%m-%d')
end = datetime.strptime(end, '%Y-%m-%d')
step = timedelta(days=1)
while start <= end:
    sol.loc[len(sol.index)] = [start]
    start += step

# Add local time
sol['Time'] = local_time

# Julian Day
base = '1900-01-01'
base = datetime.strptime(base, '%Y-%m-%d')
sol['Julian_Day'] = (sol['Date'] - base + dt.timedelta(
    days=2)).astype('timedelta64[D]') + 2415018.5+time_to_decimal(local_time)-utc/24

# Julian Century
sol['Julian_Century'] = (
    sol['Julian_Day']-2451545)/36525

# Geom Mean Long Sun (deg)
sol['Geom_Mean_Long_Sun(deg)'] = (280.46646+sol['Julian_Century']*(
    36000.76983 + sol['Julian_Century']*0.0003032)) % 360

# Geom Mean Anom Sun (deg)
sol['Geom_Mean_Anom_Sun(deg)'] = 357.52911+sol['Julian_Century']*(
    35999.05029 - 0.0001537*sol['Julian_Century'])

# Eccent Earth Orbit
sol['Eccent_Earth_Orbit'] = 0.016708634-sol['Julian_Century'] * \
    (0.000042037+0.0000001267*sol['Julian_Century'])

# Sun Eq of Ctr
sol['Sun_Eq_of_Ctr'] = np.sin(np.radians(sol['Geom_Mean_Anom_Sun(deg)'])
                       )*(1.914602-sol['Julian_Century']*(0.004817+0.000014
                        *sol['Julian_Century']))+np.sin(np.radians(2*sol['Geom_Mean_Anom_Sun(deg)']
                        ))*(0.019993-0.000101*sol['Julian_Century'])+np.sin(np.radians(3*
                        sol['Geom_Mean_Anom_Sun(deg)']))*0.000289

# Sun True Long (deg)
sol['Sun_True_Long(deg)'] = sol['Sun_Eq_of_Ctr'] + \
    sol['Geom_Mean_Long_Sun(deg)']

# Sun True Anom (deg)
sol['Sun_True_Anom(deg)'] = sol['Sun_Eq_of_Ctr'] + \
    sol['Geom_Mean_Anom_Sun(deg)']

# Sun Rad Vector (AUs)
sol['Sun_Rad_Vector(AUs)'] = (1.000001018*(1-sol['Eccent_Earth_Orbit']*sol['Eccent_Earth_Orbit'])
                              )/(1+sol['Eccent_Earth_Orbit']*np.cos(np.radians(sol['Sun_True_Anom(deg)'])))

# Sun App Long(deg)
sol['Sun_App_Long(deg)'] = sol['Sun_True_Long(deg)']-0.00569 - \
    0.00478*np.sin(np.radians(125.04-1934.136*sol['Julian_Century']))

# Mean Obliq Ecliptic (deg)
sol['Mean_Obliq_Ecliptic(deg)'] = 23+(26+((21.448-sol['Julian_Century']*(
    46.815+sol['Julian_Century']*(0.00059-sol['Julian_Century']*0.001813))))/60)/60

# Obliq Corr(deg)
sol['Obliq_Corr(deg)'] = sol['Mean_Obliq_Ecliptic(deg)']+0.00256 * \
    np.cos(np.radians(125.04-1934.136*sol['Julian_Century']))

# Sun Rt Ascen(deg)
sol['Sun_Rt_Ascen(deg)'] = np.degrees(np.arctan2(np.cos(np.radians(sol['Obliq_Corr(deg)']))
                        * np.sin(np.radians(sol['Sun_App_Long(deg)'])), np.cos(np.radians(sol['Sun_App_Long(deg)']))))

# Sun Declin(deg)
sol['Sun_Declin(deg)'] = np.degrees(np.arcsin(np.sin(np.radians(
    sol['Obliq_Corr(deg)']))*np.sin(np.radians(sol['Sun_App_Long(deg)']))))

# var y
sol['var_y'] = np.tan(np.radians(sol['Obliq_Corr(deg)']/2)) * \
    np.tan(np.radians(sol['Obliq_Corr(deg)']/2))

# Eq of Time(minutes)
sol['Eq_of_Time(min)'] = 4*np.degrees(sol['var_y']*np.sin(2*np.radians(sol['Geom_Mean_Long_Sun(deg)']))
                            -2*sol['Eccent_Earth_Orbit']*np.sin(np.radians(sol['Geom_Mean_Anom_Sun(deg)']))
                            +4*sol['Eccent_Earth_Orbit']*sol['var_y']*np.sin(np.radians(
                            sol['Geom_Mean_Anom_Sun(deg)']))*np.cos(2*np.radians(sol['Geom_Mean_Long_Sun(deg)']))
                            -0.5*sol['var_y']*sol['var_y']*np.sin(4*np.radians(sol['Geom_Mean_Long_Sun(deg)']))
                            -1.25*sol['Eccent_Earth_Orbit']*sol['Eccent_Earth_Orbit']*np.sin(2*np.radians(
                            sol['Geom_Mean_Anom_Sun(deg)'])))

# HA Sunrise(deg)
sol['HA_Sunrise(deg)'] = np.degrees(np.arccos(np.cos(math.radians(90.833))/(np.cos(math.radians(Latitude))
                        *np.cos(np.radians(sol['Sun_Declin(deg)'])))-np.tan(math.radians(Latitude))
                        *np.tan(np.radians(sol['Sun_Declin(deg)']))))

# Solar Noon(LST)
sol['Solar_Noon(LST)'] = (720-4*Longitude-sol['Eq_of_Time(min)']+utc*60)/1440
sol['Solar_Noon(LST)'] = [decimal_to_time(x) for x in sol['Solar_Noon(LST)']]

# Sunrise Time(LST)
sol['Sunrise_Time(LST)'] = [time_to_decimal(x) for x in sol['Solar_Noon(LST)']]
sol['Sunrise_Time(LST)'] = (sol['Sunrise_Time(LST)']*1440-sol['HA_Sunrise(deg)']*4)/1440
sol['Sunrise_Time(LST)'] = [decimal_to_time(x) for x in sol['Sunrise_Time(LST)']]

# Sunset Time(LST)
sol['Sunset_Time(LST)'] = [time_to_decimal(x) for x in sol['Solar_Noon(LST)']]
sol['Sunset_Time(LST)'] = (sol['Sunset_Time(LST)']*1440+sol['HA_Sunrise(deg)']*4)/1440
sol['Sunset_Time(LST)'] = [decimal_to_time(x) for x in sol['Sunset_Time(LST)']]

# Sunlight Duration(min)
sol['Sunlight_Duration(min)'] = sol['HA_Sunrise(deg)']*8

# Dark Duration(min)
sol['Dark_Duration(min)'] = 24*60 - sol['Sunlight_Duration(min)']

# True Solar Time(min)
sol['True_Solar_Time(min)'] = (time_to_decimal(local_time)*1440+sol['Eq_of_Time(min)']+4*Longitude-60*utc) % 1440

# Hour Angle(deg)
# if sol['True_Solar_Time(min)']/4<0 then ['True_Solar_Time(min)']/4-180
sol['Hour_Angle(deg)'] = sol['True_Solar_Time(min)'].apply(lambda x: (x/4+180) if (x/4)<0 else (x/4-180))

# Solar Zenith Angle(deg)
sol['Solar_Zenith_Angle(deg)'] = np.degrees(np.arccos(math.sin(math.radians(Latitude))*np.sin(np.radians(sol['Sun_Declin(deg)']))
                        +math.cos(math.radians(Latitude))*np.cos(np.radians(sol['Sun_Declin(deg)']))*np.cos(np.radians(
                            sol['Hour_Angle(deg)']))))

# Solar Elevation Angle(deg)
sol['Solar_Elevation_Angle(deg)'] = 90 - sol['Solar_Zenith_Angle(deg)']

# Approx Atmospheric Refraction                                                                                        (-20.772/np.tan(np.radians(z)))))))))/3600
sol['Approx_Atomospheric_Refraction'] = np.nan
sol.loc[sol['Solar_Elevation_Angle(deg)'] > 85, 'Approx_Atomospheric_Refraction'] = 0
sol.loc[(sol['Solar_Elevation_Angle(deg)'] <= 85) & (sol['Solar_Elevation_Angle(deg)'] > 5), 'Approx_Atomospheric_Refraction'] = 58.1/np.tan(np.radians(sol['Solar_Elevation_Angle(deg)']))-0.07/pow(np.tan(np.radians(sol['Solar_Elevation_Angle(deg)'])),3) + 0.000086/pow(np.tan(np.radians(sol['Solar_Elevation_Angle(deg)'])),5)
sol.loc[(sol['Solar_Elevation_Angle(deg)'] <= 5) & (sol['Solar_Elevation_Angle(deg)'] > -0.575), 'Approx_Atomospheric_Refraction'] = 1735+sol['Solar_Elevation_Angle(deg)']*(-518.2+sol['Solar_Elevation_Angle(deg)']*(103.4+sol['Solar_Elevation_Angle(deg)']*(-12.79+sol['Solar_Elevation_Angle(deg)']*0.711)))
sol.loc[sol['Solar_Elevation_Angle(deg)'] <= -0.575, 'Approx_Atomospheric_Refraction'] = -20.772/np.tan(np.radians(sol['Solar_Elevation_Angle(deg)']))
sol['Approx_Atomospheric_Refraction'] = sol['Approx_Atomospheric_Refraction']/3600

# Solar Elevation Corrected (deg)
sol['Solar_Elevation_corr(deg)'] = sol['Solar_Elevation_Angle(deg)'] + sol['Approx_Atomospheric_Refraction']

# Solar Azimuth Angle (deg cw from N)
sol['Solar_Azimuth_Angle(deg_cw_from_N)'] = np.nan
sol.loc[sol['Hour_Angle(deg)'] > 0, 'Solar_Azimuth_Angle(deg_cw_from_N)'] = (np.degrees(np.arccos(((math.sin(math.radians(Latitude))*np.cos(np.radians(sol['Solar_Zenith_Angle(deg)'])))-np.sin(np.radians(sol['Sun_Declin(deg)'])))/(math.cos(math.radians(Latitude))*np.sin(np.radians(sol['Solar_Zenith_Angle(deg)'])))))+180) % 360
sol.loc[sol['Hour_Angle(deg)'] <= 0, 'Solar_Azimuth_Angle(deg_cw_from_N)'] = (540-np.degrees(np.arccos(((math.sin(math.radians(Latitude))*np.cos(np.radians(sol['Solar_Zenith_Angle(deg)'])))-np.sin(np.radians(sol['Sun_Declin(deg)'])))/(math.cos(math.radians(Latitude))*np.sin(np.radians(sol['Solar_Zenith_Angle(deg)'])))))) % 360

# Account for daylight savings
if daylight_savings.lower() == 'yes':
    start = datetime.strptime(start_day, '%Y-%m-%d')
    end = datetime.strptime(end_day, '%Y-%m-%d')
    sol.loc[(sol['Date'] <= end) & (sol['Date'] >= start), 'Solar_Noon(LST)'] = [tz_offset(x, 1) for x in sol.loc[(sol['Date'] <= end) & (sol['Date'] >= start), 'Solar_Noon(LST)']]
    sol.loc[(sol['Date'] <= end) & (sol['Date'] >= start), 'Sunrise_Time(LST)'] = [tz_offset(x, 1) for x in sol.loc[(sol['Date'] <= end) & (sol['Date'] >= start), 'Sunrise_Time(LST)']]
    sol.loc[(sol['Date'] <= end) & (sol['Date'] >= start), 'Sunset_Time(LST)'] = [tz_offset(x, 1) for x in sol.loc[(sol['Date'] <= end) & (sol['Date'] >= start), 'Sunset_Time(LST)']]
else:
    pass

sol.set_index('Date', inplace=True)

# Save to csv
if daylight_savings.lower() == 'yes':
    daylight_file = 'Daylight'
else:
    daylight_file = 'NoDaylight'
filename = 'Solar_' + location + '_' + str(Latitude) + '_' + str(Longitude) + '_' + daylight_file + '.csv'
sol.to_csv(filename, index=True)

### Hourly Averages or Other Weather Data
Currently the program uses U.S. Hourly Climate Normals based on 30-year averages from 1981 to 2010 to calculate average weather values in addition to Photosynthetic Photon Flux. Other weather data can be substituted or weather data can be left off all together.

Read in 30-year average weather data. The current example uses data for Salt Lake City.

In [ ]:
filename = hourly_averages_filename + '.csv'
hour_ave = pd.read_csv(filename)

Convert to datetime formats for indexing.

In [ ]:
if hourly_averages.lower() == 'yes':
    hour_ave['DATE']=pd.to_datetime(hour_ave['DATE'], format='%m-%dT%H:%M:%S')
    hour_ave['DATE']=hour_ave['DATE'].apply(lambda x: x.replace(year = year))
    hour_ave.set_index('DATE', inplace=True)
# elif daily_averages.lower() == 'yes':
#     pass
else:
    averages_file = 'NoWeather'

### Web Scraper
Accesses the [Apogee Instrument's Clear Sky Calculator](http://clearskycalculator.com/quantumsensor.htm) to calculate Phososynthetic Photon Flux density.

Create an instance of the web driver and open site.

In [ ]:
wd = webdriver.Chrome()
wd.get('http://clearskycalculator.com/quantumsensor.htm')

Setup locations to fill on webpage through Selenium interface.

In [ ]:
Latitude_Input = wd.find_element_by_xpath('//*[@id="p1G6"]')
Longitude_Input = wd.find_element_by_xpath('//*[@id="p1G8"]')
Longitude_tz_Input = wd.find_element_by_xpath('//*[@id="p1G10"]')
Elevation_Input = wd.find_element_by_xpath('//*[@id="p1G12"]')
Day_Input = wd.find_element_by_xpath('//*[@id="p1G14"]')
Time_Input = wd.find_element_by_xpath('//*[@id="p1G16"]')
Daylight_Savings_Input = wd.find_element_by_xpath('//*[@id="p1G18"]')
Air_Temperature_Input = wd.find_element_by_xpath('//*[@id="p1G20"]')
Relative_Humidity = wd.find_element_by_xpath('//*[@id="p1G22"]')
Recalculate = wd.find_element_by_xpath('//*[@id="panel1"]/table/tbody/tr[23]/td[3]/input')
Estimated_PPF = wd.find_element_by_xpath('//*[@id="p1L6"]')

Loops through all days and hours in a year to estimate PPF.

In [ ]:
#Initial inputs that set location for Apogee Scientific model
Latitude_Input.send_keys(Keys.CONTROL,"a")
Latitude_Input.send_keys(str(Latitude))

Longitude_Input.send_keys(Keys.CONTROL,"a")
Longitude_Input.send_keys(str(Longitude * -1))

Longitude_tz_Input.send_keys(Keys.CONTROL,"a")
Longitude_tz_Input.send_keys(str(Longitude_tz))

Elevation_Input.send_keys(Keys.CONTROL,"a")
Elevation_Input.send_keys(str(Elevation))

Daylight_Savings_Input.send_keys(Keys.CONTROL,"a")
if daylight_savings.lower() == 'yes':
    Daylight_Savings_Input.send_keys('1')
else:
    Daylight_Savings_Input.send_keys('0')
    
#Create an array with headers to store values
if hourly_averages.lower() == 'yes':
    # Creates headers for photoperiod length, PPF, and Hourly Normals
    weather_model = pd.DataFrame(columns = ['Date','Light(hours)','Dark(hours)','Estimated_PPF(umol/m^2*s^1)','Temperature(90th)','Temperature(mean)','Temperature(10th)','DewPoint(90th)','DewPoint(mean)','DewPoint(10th)','CloudsBroken','CloudsClear','CloudsScattered','CloudsOvercast','CloudsFew','RelativeHumidity'])
else:
    # Creates headers for photoperiod length and PPF
    weather_model = pd.DataFrame(columns = ['Date','Light(hours)','Dark(hours)','Estimated_PPF(umol/m^2*s^1)'])

for day in range(1,366):
    #Create datetime variable from day number
    date = dt.datetime(year, 1, 1) + dt.timedelta(day - 1)
    
    #Find light and dark durations
    Light = sol.loc[date]['Sunlight_Duration(min)']
    Dark = sol.loc[date]['Dark_Duration(min)']
    
    #Input webpage values
    Day_Input.send_keys(Keys.CONTROL,"a")
    Day_Input.send_keys(day)
    
    for hour in range(0,24):
        #Convert hours to datetime format and add to date from first loop
        hour = dt.timedelta(hours = hour)
        date_time = date + hour
        
        if hourly_averages.lower() == 'yes':
            # Incorporates Hourly Normals
            try:
                # Calculates using existing values
                Hourly_Normals(date_time, hour_ave, weather_model)
            except:
                # Calulates using previous normals for gaps in data
                try:
                    offset = timedelta(days = -1)
                    Hourly_Normals_Offset(date_time, hour_ave, weather_model, offset)
                except:
                    try:
                        offset = timedelta(days = 1)
                        Hourly_Normals_Offset(date_time, hour_ave, weather_model, offset)
                    except:
                        try:
                            offset = timedelta(hours = -1)
                            Hourly_Normals_Offset(date_time, hour_ave, weather_model, offset)
                        except:
                            offset = timedelta(hours = 1)
                            Hourly_Normals_Offset(date_time, hour_ave, weather_model, offset)
        elif daily_averages.lower() == 'yes':
            # Incorporates Daily Normals
            pass
        else:
            # Ignores Hourly Normals and assumes average values for temperature and humidity
            pass

# Format index of weather model
weather_model['Date']=pd.to_datetime(weather_model['Date'], format='%Y-%m-%d %H:%M:%S')
weather_model.set_index('Date', inplace=True)

# Fill NaN values with 0
weather_model.fillna(value=0,inplace=True)

# Save weather model to csv file
if daylight_savings.lower() == 'yes':
    daylight_file = 'Daylight'
else:
    daylight_file = 'NoDaylight'

if hourly_averages.lower() == 'yes':
    averages_file = 'Hourly_' + hourly_location_averages
# elif daily_averages.lower() == 'yes':
#     averages_file = 'Daily_' + daily_location_averages
else:
    averages_file = 'NoWeather'
    
filename = 'PPF_' + location + '_' + str(Latitude) + '_' + str(Longitude) + '_' + daylight_file + '_' + averages_file + '.csv'
weather_model.to_csv(filename, index=True)